## metaclass 超越变形特性

-   yaml
    -   优点:完全不需要提前知道任何类型信息，让超动态配置编程成了可能
    -   YAML 的动态序列化 / 逆序列化是由 metaclass 实现
        -   利用 YAMLObjectMetaclass 的**init**方法，为所有 YAMLObject 子类执行 add_constructor()
        -   YAML 应用 metaclass，拦截了所有 YAMLObject 子类的定义 `cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)`



In [ ]:
class Monster(yaml.YAMLObject):
    yaml_tag = u'!Monster'

    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks

    def __repr__(self):
        return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
            self.__class__.__name__, self.name, self.hp, self.ac,
            self.attacks)


yaml.load("""
--- !Monster
name: Cave spider
hp: [2,6]    # 2d6
ac: 16
attacks: [BITE, HURT]
""", Loader=yaml.FullLoader)


In [ ]:

print(yaml.dump(Monster(
    name='Cave lizard', hp=[3, 6], ac=16, attacks=['BITE', 'HURT'])))

## Python 底层语言设计层面是如何实现 metaclass 的？

-  所有的 Python 的用户定义类都是 type 类的实例

In [17]:
# 类本身是 type 类的实例
class MyClass:
    pass


instance = MyClass()

print(type(MyClass))


<class 'type'>



- 用户自定义类，只不过是 type 类的__call__运算符重载 `class = type(classname, superclasses, attributedict)`
-   metaclass 仅仅是给小部分 Python 开发者在开发框架层面的 Python 库时使用的。在应用层，metaclass 往往不是很好的选择

In [18]:
class MyClass:
    data = 1


instance = MyClass()
MyClass, instance


(__main__.MyClass, <__main__.MyClass at 0x10bf5be20>)

In [19]:
MyClass = type('MyClass', (), {'data': 1})
instance = MyClass()
MyClass, instance


(__main__.MyClass, <__main__.MyClass at 0x10bf5b220>)

-  metaclass 是 type 的子类，通过替换 type 的 `__call__` 运算符重载机制，“超越变形”正常的类 `class = MyMeta(classname, superclasses, attributedict)`

In [20]:
class Mymeta(type):
    def __init__(self, name, bases, dic):
        super().__init__(name, bases, dic)
        print('===>Mymeta.__init__')
        print(self.__name__)
        print(dic)
        print(self.yaml_tag)

    def __new__(cls, *args, **kwargs):
        print('===>Mymeta.__new__')
        print(cls.__name__)
        return type.__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('===>Mymeta.__call__')
        obj = cls.__new__(cls)
        cls.__init__(cls, *args, **kwargs)
        return obj


class Foo(metaclass=Mymeta):
    yaml_tag = '!Foo'

    def __init__(self, name):
        print('Foo.__init__')
        self.name = name

    def __new__(cls, *args, **kwargs):
        print('Foo.__new__')
        return object.__new__(cls)


foo = Foo('foo')


===>Mymeta.__new__
Mymeta
===>Mymeta.__init__
Foo
{'__module__': '__main__', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x10bedf040>, '__new__': <function Foo.__new__ at 0x10bedf550>}
!Foo
===>Mymeta.__call__
Foo.__new__
Foo.__init__


In [ ]:
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)


class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')


class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')


class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings  # 保存属性和列的映射关系
        attrs['__table__'] = name  # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)


class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (
            self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))
